In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format

In [2]:
kernel = 0.5 * RBF(length_scale=1, length_scale_bounds=(1, 20)) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,0.5)) 

In [3]:
X_ = np.linspace(-20, 50, 70 + 1)
wave = np.linspace(2000, 9200, 720 + 1)
save_path = '/home/joao/Documentos/PCA'

In [4]:
file = open("names.txt")
lines = file.readlines()
file.close()

In [5]:
training_set = []

In [6]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [7]:
training_set

['SN2012fr',
 'SN2005cf',
 'SN2001V',
 'SN2003du',
 'SN2011fe',
 'SN2014J',
 'SN1994D',
 'SN2004dt',
 'SN2012dn',
 'SN2002bo',
 'SN1998aq',
 'SN2007le',
 'SN2007af',
 'SN1999aa',
 'SN2000cx',
 'SN2005hk',
 'SN2003cg',
 'SN2009dc',
 'SN1999ac',
 'SN1999by',
 'SN2008ar',
 'SN2005M',
 'SN2001ep']

In [8]:
for j in range(0,len(training_set)):

    list_data = [[[],[]] for y in range(0,721)]
    
    sn_names=glob.glob("/home/joao/Documentos/templates/data_ext/"+training_set[j]+"*"+".dat")
    #sn_names=glob.glob("/home/joao/Documentos/templates/data_ext/"+"SN2014J"+"*"+".dat")
    #print(sn_names)
    
    for p in range(0,len(sn_names)):

        sn_names[p] = sn_names[p][41:]
    
    
    print(training_set[j])
    
    #print(sn_names)
    
    if len(sn_names) > 3:
        
        for i in range(0,len(sn_names)):

            #y = []
            z = []

            file = open(sn_names[i])
            lines = file.readlines()
            file.close()

            for line in lines:
                if line.startswith('#'): continue
                co=line.rstrip().replace('INDEF','Nan').split()

                #y.append(co[0])
                z.append(co[1])
        

            #y = np.array(y, dtype=float)
            z = np.array(z, dtype=float)
            #print(z)
            
            
            for l in range(0,len(z)):
                
                if z[l] < 10**(-64):
                    
                     z[l] = 0
            
            
            timetemp = lines[0]
            timetemp = timetemp[:-1]
            timetemp = float(timetemp[1:])
            
            for jj in range(0,len(z)):
                
                list_data[jj][0].append([timetemp])
                list_data[jj][1].append(z[jj])

            #print(i)
            
        Export = []

        for jj in range(0,len(list_data)):

            teste = all(v == 0 for v in list_data[jj][1])
            
            if teste == False:
                #print(list_data[jj][1])
                ff = max(list_data[jj][1])
                #print(ff)
                list_data[jj][1] = list_data[jj][1]/ff
                
                #print(list_data[jj][0])
                #print(list_data[jj][1])
                
                #gp = GaussianProcessRegressor(kernel=kernel, alpha=np.var(list_data[jj][1])).fit(list_data[jj][0], list_data[jj][1])
                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(list_data[jj][0], list_data[jj][1])
                
                #val_score = gp.score(yy, zz)
                #score.append(val_score)

                y_mean = gp.predict(X_[:, np.newaxis])
                
                for l in range(0,len(y_mean)):

                    Export.append(y_mean[l]*ff)
                
            else:
                
                y_mean = [0 for y in range(0,71)]
                #print(len(y_mean), len(X_))
                
                for l in range(0,len(y_mean)):

                    Export.append(y_mean[l])
          
        table = []
        for l in range(0,len(Export)):
            table.append((Export[l],training_set[j]))


        name_of_file = training_set[j]

        completeName = os.path.join(save_path, name_of_file+".dat")

        f = open(completeName, 'w')
        f.write(tabulate(table, tablefmt="plain"))
        f.close()
     

SN2012fr
SN2005cf
SN2001V
SN2003du
SN2011fe
SN2014J
SN1994D
SN2004dt
SN2012dn
SN2002bo
SN1998aq
SN2007le
SN2007af
SN1999aa
SN2000cx
SN2005hk
SN2003cg
SN2009dc
SN1999ac
SN1999by
SN2008ar
SN2005M
SN2001ep
